## Final version pitch timestamper

This is the working version of the pitch timestamper. I have different working versions and iterations in the sandbox notebook. 

This routine takes in a image of the score infographic and finds it via template matching for each frame of the video.
The video is over 3 hours long. To cut down on processing time only one frame is grabbed per second of video. 
After earch pitch is thrown, the top of the infographic is altered to report the speed of the pitch, this is used as the trigger to timestamp for a pitch. 
A p that is obscured by the speed is what a second template match is focused on. When it is obscured the program logs the time.
Tesseract is used to then pull the pitcher name and pitch count from the infographic. This info is used to match up with the baseball savant logs that were pulled that have the type of pitch and outcome of each pitch reported. By joining the two logs, I will have pitch type and outcome connected to the timestamp of each pitch visual and audio.


Some tinkering will be requried for every video I use. Hopefully minimal when it comes to tesseract. But I can for see day versus night games forcing changes to the image processing parameters.

Second, this program was written using a fox sports broadcast. Different broadcasts have different infographics. Differnt infographics mean different ways to trigger the timestamp. Again the hope is they will be minimal. Fox sports has deals with I believe a vast majority of regular season broadcasting rights. One I can think of is Yankees YES network will be different, but I checked and it also flashes the speed of each pitch after its thrown and can be used as a trigger. 

In [1]:
%%time
#MONITOR PROCESSING TIME


#IMPORT LIBRARIES
from IPython.display import clear_output
try:
    from PIL import Image
except ImportError:
    import Image

import cv2
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

import numpy as np
import pandas as pd


path='E://HOU18/games/game 167/'



#INITIALIZE VARIABLES
h=0
#count instances of finding a pitch and save info in list
got=0
lst=[]

vertex_x=[]
vertex_y=[]

#loop variable how many consecutive frames is the pitch count p being detected and how many frames is required before grabbing info
consecutive=0
threshold=4

# frames to skip per loop
n=0
skip=30

# config options for tesseract
pitchconfig="-l eng --psm 8 --oem 3 -c tessedit_char_blacklist=!@#$%^&*()_-+=|\<>:;{}[]|\,.?/1234567890"
countconfig='-l eng --psm 8 --oem 3 -c tessedit_char_whitelist=0123456789'

#IMPORT TEMPLATES AND GRAB DIMENSIONS
box = cv2.imread('E:/HOU18/snapshots/tbs box.jpg')  
height, width,chan = box.shape

p=cv2.imread('E:/HOU18/snapshots/tbs p.jpg') 
height_p, width_p,chan = p.shape

#IMPORT VIDEO
cap = cv2.VideoCapture(f'{path}game 167 video.mp4')


if cap.isOpened()== False: 
    print("Error opening the video file. Please double check your file path for typos")

## FIRST LOOP FINDS INFOGRAPHIC POSITION
cap.set(cv2.CAP_PROP_POS_FRAMES, 38000)
while cap.isOpened():
      
    
    #frame skip to middle of game where infographic will be 30 frame per second, say we want to go to 15minutes into the game
    #15*60*30 =27000
    
    
    # Read a new frame
    ret, frame = cap.read()

    #check if frame grab was a success
    if ret == True and cap.get(cv2.CAP_PROP_POS_FRAMES)<40000:
        
        # FIND INFOGRAPHIC AND MAKE VERTICIES FOR RECTANGLE
        res = cv2.matchTemplate(frame,box,cv2.TM_CCOEFF_NORMED)
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
        
        top_left=max_val
        
        top_left = max_loc
        bottom_right = (top_left[0] + width, top_left[1] + height)
        
        cv2.rectangle(frame,top_left, bottom_right,(0,200,200), 3)
        
        vertex_x.append(max_loc[0])
        vertex_y.append(max_loc[1])
        
                #DISPLAY FRAME
        cv2.imshow('frame', frame)
        
        # Exit if ESC pressed
        k = cv2.waitKey(1) & 0xff
        if k == 27 : 
            break
            
    else:
        break
 
#TIDY UP
cap.release()
cv2.destroyAllWindows()


#collect top left vertex position of template match for each frame. The value that occurs most often should be our fixed pixel.
pixel_x = max(set(vertex_x), key = vertex_x.count)
pixel_y = max(set(vertex_y), key = vertex_y.count) 

print(pixel_x)
print(pixel_y)

41
25
Wall time: 5min 16s


In [2]:
%%time
#MONITOR PROCESSING TIME

#IMPORT LIBRARIES
from IPython.display import clear_output
try:
    from PIL import Image
except ImportError:
    import Image

import datetime
import re
import cv2
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

import numpy as np
import pandas as pd

#pixel location of box infographic 
#fox: 1014 498
#at&t: 838 611
#tbs: 41 23
#nbc: 62 35
pixel_x=1014
pixel_y=498

#loop variable how many consecutive frames is the pitch count p being detected and how many frames is required before grabbing info
threshold=4

# config options for tesseract
pitchconfig="-l eng --psm 8 --oem 3 -c tessedit_char_blacklist=!@#$%^&*()_-+=|\<>:;{}[]|\,.?/1234567890—é"
countconfig='-l eng --psm 6 --oem 3 -c tessedit_char_whitelist=0123456789'

#IMPORT TEMPLATES AND GRAB DIMENSIONS
box = cv2.imread('E:/HOU18/snapshots/full box.jpg')  
height, width,chan = box.shape

p=cv2.imread('E:/HOU18/snapshots/pitch P.jpg') 
height_p, width_p,chan = p.shape

game_set=[135]

for x in game_set:
    consecutive=0
    # frames to skip per loop
    n=0
    skip=30

    #INITIALIZE VARIABLES
    miss=0
    
    #count instances of finding a pitch and save info in list
    got=0
    lst=[]
    
    path=f'E://HOU18/games/game {x}/'

    #IMPORT VIDEO
    cap = cv2.VideoCapture(f'{path}game {x} video.mp4')

    if cap.isOpened()== False: 
        print("Error opening the video file. Please double check your file path for typos")


    while cap.isOpened():

        #frame skip
        cap.set(cv2.CAP_PROP_POS_FRAMES, n)

        # Read a new frame
        ret, frame = cap.read()

        #check if frame grab was a success
        if ret == True:

            cropFrameInfographic= frame[pixel_y:(pixel_y+height),pixel_x:(pixel_x+width)]
            #convert frame to RGB for tesseract
            
            tessFrame = cv2.cvtColor(cropFrameInfographic, cv2.COLOR_BGR2RGB)
            
            #FIND PITCH COUNT P IN INFOGRAPHIC AND MAKE VERTICIES FOR RECTANGLE
            res_p =cv2.matchTemplate(cropFrameInfographic,p,cv2.TM_CCOEFF_NORMED)
            min_val_p, max_val_p, min_loc_p, max_loc_p = cv2.minMaxLoc(res_p)

            top_left_p = max_loc_p
            bottom_right_p = (top_left_p[0] + width_p, top_left_p[1] + height_p) 

            #DETECT P IN CORRECT LOCATION
            #print(top_left_p)
                
            # fox
            if top_left_p[0] >= 139 and top_left_p[0] <=154 and top_left_p[1] >=0 and top_left_p[1] <=33 :
            
            # at&t  
            #if top_left_p[0]==217 or top_left_p[0]==24:
            
            #tbs
            #if top_left_p[0]==173 or top_left_p[0]==178:
                consecutive+=1
                
                #DRAW INFOGRAPHIC AND P BOX RECTANGLES
                #P BOX
                cv2.rectangle(cropFrameInfographic,top_left_p, (bottom_right_p[0],bottom_right_p[1]),(0,255,0), 1)
                # if consecutive frames have the p in correct location extract info
                if consecutive==threshold:
                    miss+=1
                    #DRAW RECTANGLES
                    #PITCHER
                    #fox:
                    cv2.rectangle(cropFrameInfographic,(0,0),(top_left_p[0],bottom_right_p[1]),(255,80,80), 3)
                    #at&t 
                    #cv2.rectangle(cropFrameInfographic,(0,0),(150,height_p+8),(255,80,80), 3)
                    #tbs
                    #cv2.rectangle(cropFrameInfographic,(0,top_left_p[1]),(top_left_p[0],bottom_right_p[1]),(255,80,80), 3)
                    
                    #PITCH COUNT
                    #fox 
                    cv2.rectangle(cropFrameInfographic,(bottom_right_p[0]-10,top_left_p[1]-10),(width,bottom_right_p[1]),(80,80,255), 1)
                    #at&t
                    #cv2.rectangle(cropFrameInfographic,(top_left_p[0]-27,3),(top_left_p[0],height_p+15),(80,80,255), 1)
                    #tbs
                    #cv2.rectangle(cropFrameInfographic,(bottom_right_p[0],top_left_p[1]),(width,height),(80,80,255), 1)
                    
                    
                    #REPORT PITCH RECORDED
                    #cv2.putText(cropFrameInfographic, "pitch recorded", (0,200), cv2.FONT_HERSHEY_SIMPLEX, 1,(0,255,0),2)

                    # GRAB PITCHER IMAGE
                    #fox 
                    cropFramePitcher= tessFrame[top_left_p[1]:bottom_right_p[1],10:top_left_p[0]]
                    #at&t 
                    #cropFramePitcher= tessFrame[0:height_p+8,0:150]
                    #tbs
                    #cropFramePitcher= tessFrame[top_left_p[1]:bottom_right_p[1],0:top_left_p[0]]
                    #PROCESS PITCHER IMAGE
                    
                    
                    cropFramePitcher= cv2.cvtColor(cropFramePitcher, cv2.COLOR_BGR2GRAY)

                    scale=600
                    w = int(cropFramePitcher.shape[1] * scale / 100)
                    h = int(cropFramePitcher.shape[0] * scale/ 100)
                    dim = (w, h)
                    cropFramePitcher=cv2.resize(cropFramePitcher,dim,cv2.INTER_AREA)

                    r,cropFramePitcher=cv2.threshold(cropFramePitcher, 105, 255, cv2.THRESH_BINARY_INV)
                    
                    cropFramePitcher=cv2.medianBlur(cropFramePitcher,7)
                    
                    #GRAB PITCH COUNT IMAGE
                    #fox
                    cropFrameCount= tessFrame[top_left_p[1]:bottom_right_p[1],bottom_right_p[0]-2:width]
                    #at&t
                    #cropFrameCount= tessFrame[3:height_p+8,top_left_p[0]-29:top_left_p[0]]
                    #tbs
                    #cropFrameCount= tessFrame[top_left_p[1]:height,bottom_right_p[0]:width]
                    
                    #PROCESS PITCH COUNT IMAGE
                    cropFrameCount= cv2.cvtColor(cropFrameCount, cv2.COLOR_BGR2GRAY)

                    w = int(cropFrameCount.shape[1] * scale / 100)
                    h = int(cropFrameCount.shape[0] * scale/ 100)
                    dim = (w, h)
                    cropFrameCount=cv2.resize(cropFrameCount,dim,cv2.INTER_AREA)

                    r,cropFrameCount=cv2.threshold(cropFrameCount, 115, 255, cv2.THRESH_BINARY_INV)

                    cropFrameCount=cv2.medianBlur(cropFrameCount,7)
                    #cropFrameCount = cv2.GaussianBlur(cropFrameCount,(5,5),0)
                    
                    #CALL TESSERACT TO CONVERT IMAGE OF TEXT TO STRING

                    pitcher=pytesseract.image_to_string(cropFramePitcher,config=pitchconfig)

                    count=pytesseract.image_to_string(cropFrameCount,config=countconfig)

                    pitcher=(re.sub('[^A-Za-z]+', '', pitcher))
                    pitcher=pitcher.lower()
                    pitcher=pitcher.capitalize()
                    #OUTPUT RESULT
                    print(f'Pitcher: {pitcher} Pitch Count: {count}')

                    #SHOW IMAGES
                    cv2.imshow('Pitcher',cropFramePitcher)
                    cv2.imshow('Count',cropFrameCount)

                    #COLLECT INFO ON FRAME
                    got+=1
                    t=cap.get(cv2.CAP_PROP_POS_MSEC)
                    t=int(np.floor(t/1000))

                    if count == '':
                        lst.append([int(got),pitcher,f'miss{miss}',t])
                    else:
                        lst.append([int(got),pitcher,count,t])
                # PERSIST THE BOXES PAST THRESHOLD
                elif consecutive > threshold:

                    #cv2.rectangle(frame,top_left_p, bottom_right_p,(0,255,0), 1)

                    #DRAW RECTANGLES
                    #fox:
                    #PITCHER
                    cv2.rectangle(cropFrameInfographic,(0,0),(top_left_p[0],bottom_right_p[1]),(255,80,80), 3)
                    #PITCH COUNT
                    cv2.rectangle(cropFrameInfographic,(bottom_right_p[0]-10,top_left_p[1]-10),(width,bottom_right_p[1]),(80,80,255), 1)

                    
                    #at&t
                    #PITCHER
                    #cv2.rectangle(cropFrameInfographic,(0,0),(150,height_p+8),(255,80,80), 3)
                    #PITCH COUNT
                    #cv2.rectangle(cropFrameInfographic,(top_left_p[0]-27,3),(top_left_p[0],height_p+15),(80,80,255), 1)
                                        
                    #tbs
                    #PITCHER
                    #cv2.rectangle(cropFrameInfographic,(0,top_left_p[1]),(top_left_p[0],bottom_right_p[1]),(255,80,80), 3)
                    #PITCH COUNT
                    #cv2.rectangle(cropFrameInfographic,(bottom_right_p[0],top_left_p[1]),(width,height),(80,80,255), 1)


                    #cv2.putText(cropFrameInfographic, "pitch recorded", (0,200), cv2.FONT_HERSHEY_SIMPLEX, 1,(0,255,0),2)

            # P NOT DETECTED RESETS COUNTER
            else:
                consecutive=0

            #DISPLAY FRAME
            cv2.imshow('frame', cropFrameInfographic)

            # Exit if ESC pressed
            k = cv2.waitKey(1) & 0xff
            if k == 27 : 
                break

        # EXIT IF FAIL TO FIND FRAME
        else:
            break

        #FRAME SKIP ITERATOR
        n= n+skip

    #TIDY UP
    clear_output(wait=True)
    cap.release()
    #cv2.destroyAllWindows()

    #WRITE INFO TO DATAFRAME
    print(f'game {x} done')
    df=pd.DataFrame(lst,columns =['Pitch number','Pitcher','Pitch Count','time'])
    df=df[~df.duplicated(subset=['Pitcher','Pitch Count'],keep='first')]

    df.to_csv(f'{path}pitch timelog.csv',index=False)

print('done')

game 135 done
done
Wall time: 23min 15s


In [ ]:
df.head()

game 2 processing time 6.15.20 45min 38s

''  '' processing time 6.16.20 22min 49s

after each game is collected certain by hand corrections are going to be run through.

1) Tesseract is not perfect, names will be close but not accurate everytime, for instance when collecting game 2, It found the pitcher Keal's name to be Kea. Similarly the numbers can be misread from time to time.

2) false postive timestamps. Simply remove these.

I have pitch logs pulled from the baseball savant website, the information pulled by tesseract is to match the timestamp pulled from the game to the pitch outcome. Therefore I have a cross reference to correct the misreads by Tesseract. I.e I know the closer was named Keal not Kea, or how many pitches each pitcher threw.

These actions are continued in the log joiner program